In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.patches as mpatches

In [ ]:
mpl.use("pgf")
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 30,
    "font.size": 20,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 18,
    "xtick.labelsize": 20,
    "ytick.labelsize": 20,
}

plt.rcParams.update(tex_fonts)




In [2]:
width = 600

def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [3]:
def moving_average(data, window):
    if window > 1:
        max_val = data.max()
        id_max_val = data.idxmax()
        rm = data.rolling(window).mean()
        # rm[id_max_val] = max_val
        return rm
    else:
        return data

### Load Data 

In [4]:
DATA_DIR ="./csv_files/v2"

CIFAR_IID = os.path.join(DATA_DIR,'fedadap_cifar10_iid_consol.csv')
CIFAR_NIID_0_1 = os.path.join(DATA_DIR,'fedadap_cifar10_niid_0_1_consol.csv')
CIFAR_NIID_0_6 = os.path.join(DATA_DIR,'fedadap_cifar10_niid_0_6_consol.csv')
CIFAR_NIID_100 = os.path.join(DATA_DIR,'fedadap_cifar10_niid_100_consol.csv')

CIFAR100_IID = os.path.join(DATA_DIR,'fedadap_cifar100_iid_consol.csv')
CIFAR100_NIID_0_1 = os.path.join(DATA_DIR,'fedadap_cifar100_niid_0_1_consol.csv')
CIFAR100_NIID_0_6 = os.path.join(DATA_DIR,'fedadap_cifar100_niid_0_6_consol.csv')
CIFAR100_NIID_100 = os.path.join(DATA_DIR,'fedadap_cifar100_niid_100_consol.csv')

ABLATION_CIFAR = os.path.join(DATA_DIR,'fedadap_ablation.csv')

SHAKESPEARE = os.path.join(DATA_DIR,'fedadap_shakespeare_consol.csv')

PLOTS_DIR = os.path.join('./','plots','v2')

## Cifar-10 

#### Non-IID

In [8]:
cifar10_niid_100_consol = pd.read_csv(CIFAR_NIID_100)
cifar10_niid_100_consol = cifar10_niid_100_consol * 100
cifar10_niid_100_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,10.380000,0.050000,10.686667,0.708543,10.916667,0.843880,10.653333,0.650026,11.816667,0.933292,11.633333,1.625003,11.710000,1.517004,11.430000,2.076247,13.386667,0.999617
1,200,12.506667,1.417651,15.590000,3.128082,10.000000,0.000000,10.000000,0.000000,13.293333,2.100508,14.036667,1.457818,13.803333,3.435613,15.170000,3.479526,28.683333,0.795760
2,300,21.800000,2.181032,21.973333,3.019558,29.653333,5.810287,29.340000,4.320637,22.610000,6.252136,22.770000,2.155157,21.176667,1.832330,22.300000,1.193776,31.560000,0.770779
3,400,35.603333,1.567716,35.676667,2.362125,37.910000,3.882113,33.006667,1.324550,38.133333,1.782087,35.603333,2.576555,36.260000,1.605989,31.890000,3.376877,36.590000,1.414602
4,500,37.926667,2.177391,38.106667,2.375170,45.560000,2.190000,44.676667,0.502129,39.630000,5.324932,39.030000,1.178177,35.513333,5.915144,35.853333,3.809689,38.373333,1.489373


In [9]:
# niid alpha = 100

window = 2
target_acc = 75

# Define the DataFrame containing your data
dataframe = cifar10_niid_100_consol
filename = 'cifar_10_fedadap_niid_100_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
ax[0].set_xticks(range(0, 201, 20))
ax[0].set_xticklabels(range(0, 201, 20), rotation=45)
ax[0].set_yticks(range(10, 77, 10))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
ax[1].set_xticks(range(0, 201, 20))
ax[1].set_xticklabels(range(0, 201, 20), rotation=45)
ax[1].set_yticks(range(10, 77, 10))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
ax[2].set_xticks(range(0, 201, 20))
ax[2].set_xticklabels(range(0, 201, 20), rotation=45)
ax[2].set_yticks(range(10, 77, 10))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
ax[3].set_xticks(range(0, 201, 20))
ax[3].set_xticklabels(range(0, 201, 20), rotation=45)
ax[3].set_yticks(range(10, 77, 10))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
ax[4].set_xticks(range(0, 221, 20))
ax[4].set_xticklabels(range(0, 221, 20), rotation=45)
ax[4].set_yticks(range(10, 77, 10))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

In [10]:
cifar10_niid_0_6_consol = pd.read_csv(CIFAR_NIID_0_6)
cifar10_niid_0_6_consol = cifar10_niid_0_6_consol * 100
cifar10_niid_0_6_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,10.000000,0.000000,10.970000,1.680089,10.006667,0.011547,12.910000,2.909656,10.220000,0.381051,10.000000,0.000000,11.226667,1.092901,10.946667,1.420469,13.136667,0.262742
1,200,9.950000,0.104403,13.470000,6.001558,10.000000,0.000000,10.000000,0.000000,11.806667,1.580011,11.070000,1.419260,10.950000,1.645448,11.773333,1.627923,19.943333,1.939106
2,300,15.030000,3.908107,12.960000,0.977804,17.963333,7.106492,21.646667,10.098695,15.696667,2.185322,14.653333,3.539835,19.110000,8.225515,19.090000,6.559886,24.330000,5.924762
3,400,19.256667,7.315069,21.490000,7.262692,15.763333,5.166646,16.923333,6.016181,20.873333,3.351154,20.903333,2.338853,22.310000,3.643172,21.980000,0.835943,30.380000,2.062692
4,500,26.596667,6.585608,26.303333,1.645063,27.403333,4.755695,29.640000,5.228422,26.436667,2.364959,25.663333,0.941612,23.190000,9.238463,24.050000,4.989749,33.270000,1.160517


In [11]:
# niid alpha = 0.6

window = 2
target_acc = 71

# Define the DataFrame containing your data
dataframe = cifar10_niid_0_6_consol
filename = 'cifar_10_fedadap_niid_0_6_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_xlim(0,221)
# ax[0].set_ylim(0.21,0.73)
ax[0].set_xticks(range(0, 221, 20))
ax[0].set_xticklabels(range(0, 221, 20), rotation=45)
ax[0].set_yticks(range(10, 73, 10))

# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
# ax[1].set_xlim(0,151)
# ax[1].set_ylim(0.21,0.73)
ax[1].set_xticks(range(0, 161, 20))
ax[1].set_xticklabels(range(0, 161, 20), rotation=45)
ax[1].set_yticks(range(10, 73, 10))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
# ax[2].set_xlim(0,221)
# ax[2].set_ylim(0.21,0.73)
ax[2].set_xticks(range(0, 221, 20))
ax[2].set_xticklabels(range(0, 221, 20), rotation=45)
ax[2].set_yticks(range(10, 73, 10))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
# ax[3].set_xlim(0,231)
# ax[3].set_ylim(0.21,0.73)
ax[3].set_xticks(range(0, 231, 20))
ax[3].set_xticklabels(range(0, 231, 20), rotation=45)
ax[3].set_yticks(range(10, 73, 10))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
# ax[4].set_xlim(0,261)
# ax[4].set_ylim(0.21,0.73)
ax[4].set_xticks(range(0, 261, 30))
ax[4].set_xticklabels(range(0, 261, 30), rotation=45)
ax[4].set_yticks(range(10, 73, 10))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

In [12]:
cifar10_niid_0_1_consol = pd.read_csv(CIFAR_NIID_0_1)
cifar10_niid_0_1_consol = cifar10_niid_0_1_consol * 100
cifar10_niid_0_1_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,11.056667,1.830200,10.473333,0.819837,11.553333,2.113417,11.003333,1.737824,10.573333,0.941559,10.086667,0.132791,11.703333,2.950260,12.716667,3.811749,12.603333,1.425845
1,200,11.856667,3.215841,10.000000,0.000000,10.000000,0.000000,10.000000,0.000000,10.000000,0.000000,10.000000,0.000000,10.000000,0.000000,10.466667,0.808290,12.670000,4.624576
2,300,10.000000,0.000000,10.000000,0.000000,12.616667,4.532200,12.796667,4.843969,12.650000,3.702148,10.000000,0.000000,12.110000,2.432098,11.870000,0.420357,14.443333,4.115851
3,400,11.050000,1.818653,10.000000,0.000000,10.110000,0.190526,11.996667,3.458328,10.313333,0.525484,10.013333,0.023094,11.006667,1.726306,11.170000,1.028056,20.496667,2.216987
4,500,10.800000,1.394310,10.053333,0.046188,10.000000,0.000000,10.000000,0.000000,10.730000,1.264397,10.553333,0.958401,12.293333,3.972170,11.930000,1.895969,21.033333,4.852796


In [13]:
# niid alpha = 0.1

window = 2
target_acc = 55

# Define the DataFrame containing your data
dataframe = cifar10_niid_0_1_consol
filename = 'cifar_10_fedadap_niid_0_1_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_xlim(0,401)
ax[0].set_ylim(10,57)
ax[0].set_xticks(range(0, 401, 40))
ax[0].set_xticklabels(range(0, 401, 40), rotation=45)
ax[0].set_yticks(range(10, 56, 10))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
ax[1].set_xlim(0,351)
ax[1].set_ylim(10,57)
ax[1].set_xticks(range(0, 351, 40))
ax[1].set_xticklabels(range(0, 351, 40), rotation=45)
ax[1].set_yticks(range(10, 56, 10))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
# ax[2].set_xlim(0,291)
ax[2].set_ylim(10,57)
ax[2].set_xticks(range(0, 291, 30))
ax[2].set_xticklabels(range(0, 291, 30), rotation=45)
ax[2].set_yticks(range(10, 56, 10))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
# ax[3].set_xlim(0,351)
ax[3].set_ylim(10,57)
ax[3].set_xticks(range(0, 361, 40))
ax[3].set_xticklabels(range(0, 361, 40), rotation=45)
ax[3].set_yticks(range(10, 56, 10))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
# ax[4].set_xlim(0,351)
ax[4].set_ylim(10,57)
ax[4].set_xticks(range(0, 361, 40))
ax[4].set_xticklabels(range(0, 361, 40), rotation=45)
ax[4].set_yticks(range(10, 56, 10))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

## Cifar100

In [17]:
cifar100_niid_100_consol = pd.read_csv(CIFAR100_NIID_100)
cifar100_niid_100_consol = cifar100_niid_100_consol * 100
cifar100_niid_100_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,0.990000,0.017321,1.016667,0.028868,1.010000,0.017321,1.086667,0.141539,1.146667,0.145029,1.146667,0.145029,0.940000,0.112694,1.003333,0.005774,1.21,0.043589
1,200,1.080000,0.098489,1.230000,0.248797,1.000000,0.000000,1.000000,0.000000,1.740000,0.187350,1.946667,0.064291,1.370000,0.524309,1.400000,0.238956,3.02,0.770779
2,300,5.520000,1.126810,6.890000,0.740000,1.000000,0.000000,1.000000,0.000000,5.266667,0.659571,4.603333,0.575529,6.503333,1.847819,8.253333,1.075376,7.49,0.795676
3,400,12.733333,0.758705,12.790000,0.108167,1.000000,0.000000,1.000000,0.000000,8.516667,0.771838,8.330000,0.400000,13.470000,1.099955,13.623333,0.210317,8.81,0.350856
4,500,13.200000,0.540093,14.833333,0.251064,14.606667,0.330051,14.900000,0.313209,9.450000,0.222711,9.546667,0.349333,14.370000,0.682862,14.270000,0.446430,9.88,0.660076


In [18]:
# niid = 100

window = 2
target_acc = 32

# Define the DataFrame containing your data
dataframe = cifar100_niid_100_consol
filename = 'cifar_100_fedadap_niid_100_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_xlim(0,83)
ax[0].set_ylim(10,33)
ax[0].set_xticks(range(0, 81, 10))
ax[0].set_xticklabels(range(0, 81, 10), rotation=45)
ax[0].set_yticks(range(10, 35, 5))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
ax[1].set_xlim(0,63)
ax[1].set_ylim(10,33)
ax[1].set_xticks(range(0, 61, 10))
ax[1].set_xticklabels(range(0, 61, 10), rotation=45)
ax[1].set_yticks(range(10, 35, 5))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
# ax[2].set_xlim(0,456)
ax[2].set_ylim(10,33)
ax[2].set_xticks(range(0, 460, 50))
ax[2].set_xticklabels(range(0, 460, 50), rotation=45)
ax[2].set_yticks(range(10, 35, 5))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
# ax[3].set_xlim(0,81)
ax[3].set_ylim(10,33)
ax[3].set_xticks(range(0, 81, 10))
ax[3].set_xticklabels(range(0, 81, 10), rotation=45)
ax[3].set_yticks(range(10, 35, 5))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
# ax[4].set_xlim(0,383)
ax[4].set_ylim(10,33)
ax[4].set_xticks(range(0, 351, 50))
ax[4].set_xticklabels(range(0, 351, 50), rotation=45)
ax[4].set_yticks(range(10, 35, 5))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

In [19]:
cifar100_niid_0_6_consol = pd.read_csv(CIFAR100_NIID_0_6)
cifar100_niid_0_6_consol = cifar100_niid_0_6_consol * 100
cifar100_niid_0_6_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,1.056667,0.073711,1.010000,0.017321,1.080000,0.138564,1.030000,0.051962,0.963333,0.089629,0.903333,0.185831,1.000000,0.000000,1.000000,0.000000,1.030000,0.036056
1,200,1.253333,0.241109,1.390000,0.532259,1.000000,0.000000,1.000000,0.000000,1.253333,0.227450,1.120000,0.120000,1.413333,0.258134,1.180000,0.105830,1.886667,0.557524
2,300,3.823333,0.996511,3.756667,1.765484,1.000000,0.000000,1.000000,0.000000,2.796667,0.485215,2.686667,0.642988,4.170000,0.600583,4.966667,1.038765,5.816667,0.814023
3,400,8.766667,1.293303,8.520000,1.134063,3.086667,3.614213,1.000000,0.000000,4.640000,0.810740,5.760000,0.360139,8.736667,1.095734,8.366667,0.717101,7.520000,0.592537
4,500,9.323333,0.380044,9.723333,0.539660,7.380000,5.548730,11.076667,0.702092,6.826667,0.635164,6.756667,0.415853,9.986667,0.677151,10.590000,0.357631,8.876667,0.652099


In [20]:
# niid = 0.6

window = 2
target_acc = 31

# Define the DataFrame containing your data
dataframe = cifar100_niid_0_6_consol
filename = 'cifar_100_fedadap_niid_0_6_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_xlim(0,111)
ax[0].set_ylim(10,33)
ax[0].set_xticks(range(0, 111, 10))
ax[0].set_xticklabels(range(0, 111, 10), rotation=45)
ax[0].set_yticks(range(10, 35, 5))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
# ax[1].set_xlim(0,91)
ax[1].set_ylim(10,33)
ax[1].set_xticks(range(0, 91, 10))
ax[1].set_xticklabels(range(0, 91, 10), rotation=45)
ax[1].set_yticks(range(10, 35, 5))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
# ax[2].set_xlim(0,421)
ax[2].set_ylim(10,33)
ax[2].set_xticks(range(0, 421, 50))
ax[2].set_xticklabels(range(0, 421, 50), rotation=45)
ax[2].set_yticks(range(10, 35, 5))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
# ax[3].set_xlim(0,130)
ax[3].set_ylim(10,33)
ax[3].set_xticks(range(0, 111, 10))
ax[3].set_xticklabels(range(0, 111, 10), rotation=45)
ax[3].set_yticks(range(10, 35, 5))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
# ax[4].set_xlim(0,280)
ax[4].set_ylim(10,33)
ax[4].set_xticks(range(0, 341, 50))
ax[4].set_xticklabels(range(0, 341, 50), rotation=45)
ax[4].set_yticks(range(10, 35, 5))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

In [21]:
cifar100_niid_0_1_consol = pd.read_csv(CIFAR100_NIID_0_1)
cifar100_niid_0_1_consol = cifar100_niid_0_1_consol * 100
cifar100_niid_0_1_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.00,0.000000,0.990000,0.017321,1.093333,0.128582,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
1,200,1.106667,0.184752,1.013333,0.015275,1.000000,0.000000,1.00,0.000000,1.053333,0.083865,0.983333,0.423360,1.373333,0.306649,1.036667,0.063509,1.823333,0.341955
2,300,1.996667,0.476900,1.843333,0.200083,1.000000,0.000000,1.00,0.000000,1.286667,0.230723,1.616667,0.151767,2.056667,0.829237,2.213333,1.309364,3.646667,0.385530
3,400,2.696667,1.023442,3.970000,1.438159,2.170000,1.155292,1.00,0.000000,2.670000,1.453823,2.563333,0.316596,3.976667,0.638775,4.156667,0.847899,5.173333,0.653095
4,500,3.923333,1.552750,4.853333,1.859417,4.053333,2.863937,2.31,2.268987,3.020000,1.694344,3.480000,0.939521,4.513333,0.914403,5.306667,0.540123,6.066667,0.451811


In [22]:
# niid = 0.1

window = 2
target_acc = 30

# Define the DataFrame containing your data
dataframe = cifar100_niid_0_1_consol
filename = 'cifar_100_fedadap_niid_0_1_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_xlim(0,191)
ax[0].set_ylim(0,32)
ax[0].set_xticks(range(0, 191, 20))
ax[0].set_xticklabels(range(0, 191, 20), rotation=45)
ax[0].set_yticks(range(0, 35, 5))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
# ax[1].set_xlim(0,100)
ax[1].set_ylim(0,32)
ax[1].set_xticks(range(0, 191, 20))
ax[1].set_xticklabels(range(0, 191, 20), rotation=45)
ax[1].set_yticks(range(0, 35, 5))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
# ax[2].set_xlim(0,185)
ax[2].set_ylim(0,32)
ax[2].set_xticks(range(0, 421, 50))
ax[2].set_xticklabels(range(0, 421, 50), rotation=45)
ax[2].set_yticks(range(0, 35, 5))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
# ax[3].set_xlim(0,130)
ax[3].set_ylim(0,32)
ax[3].set_xticks(range(0, 211, 20))
ax[3].set_xticklabels(range(0, 211, 20), rotation=45)
ax[3].set_yticks(range(0, 35, 5))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
# ax[4].set_xlim(0,280)
ax[4].set_ylim(0,32)
ax[4].set_xticks(range(0, 351, 50))
ax[4].set_xticklabels(range(0, 351, 50), rotation=45)
ax[4].set_yticks(range(0, 35, 5))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')

## shakespeare data plots

In [23]:
shakespeare_consol = pd.read_csv(SHAKESPEARE)
shakespeare_consol = shakespeare_consol * 100
shakespeare_consol.head()

,round,fedavg_acc,fedavg_acc_std,fedavg+fedadap_acc,fedavg+fedadap_acc_std,fedavgm_acc,fedavgm_acc_std,fedavgm+fedadap_acc,fedavgm+fedadap_acc_std,fedprox_acc,fedprox_acc_std,fedprox+fedadap_acc,fedprox+fedadap_acc_std,fedlaw_acc,fedlaw_acc_std,fedlaw+fedadap_acc,fedlaw+fedadap_acc_std,fedex_acc,fedex_acc_std
0,100,18.708678,0.088943,18.658130,0.001390,18.658290,0.001668,18.706913,0.085885,18.657327,0.000000,18.657327,0.000000,18.658771,0.002502,18.657327,0.000000,18.707716,0.087275
1,200,25.640125,2.385140,25.649432,2.455637,25.687303,2.532017,25.614770,2.434919,20.582194,1.080260,19.760736,1.605274,26.913794,2.608059,26.731016,2.581899,25.978561,2.200742
2,300,31.759901,2.559957,31.905450,1.995776,32.342256,1.445785,32.311926,1.907953,26.233552,0.867733,25.700302,0.992931,31.968515,2.831479,31.804352,2.858385,32.336318,1.639454
3,400,36.057674,1.048259,35.674145,1.304378,35.868637,0.913678,35.820335,1.081475,29.169555,0.307434,28.748636,0.528164,35.199307,1.734577,35.058091,1.839194,36.110630,1.094143
4,500,38.009179,1.453690,37.855607,1.655076,38.348257,1.414696,38.220682,1.408116,30.922556,0.240875,30.408242,0.302808,37.318987,1.583707,37.195102,1.623567,38.053630,1.416790


In [24]:


window = 2
target_acc = 51

# Define the DataFrame containing your data
dataframe = shakespeare_consol
filename = 'shakespeare_niid_performance_std'

png_filename = filename+'.png'
pdf_filename = filename+'.pdf'
pgf_filename = filename+'.pgf'


fig, ax = plt.subplots(1, 5, figsize=(24, 4), constrained_layout=True)

# FedAvg plot
ax[0].plot(moving_average(dataframe['fedavg_acc'], window), color='g', label='FedAvg', linestyle='--')
ax[0].fill_between(
    range(len(dataframe['fedavg_acc'])),
    moving_average(dataframe['fedavg_acc'] - dataframe['fedavg_acc_std'], window),
    moving_average(dataframe['fedavg_acc'] + dataframe['fedavg_acc_std'], window),
    color='g', alpha=0.2
)
ax[0].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[0].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[0].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[0].legend(loc='lower right')
ax[0].set_title('(a)')
# ax[0].set_ylim(20,52)
ax[0].set_xticks(range(0, 51, 10))
ax[0].set_xticklabels(range(0, 51, 10), rotation=45)
ax[0].set_yticks(range(20, 55, 5))
# ax[0].set_xlabel('Rounds')
ax[0].grid(True)
# ax[0].set_ylabel('Accuracy (%)')

# FedAvgm plot
ax[1].plot(moving_average(dataframe['fedavgm_acc'], window), color='g', label='FedAvgM', linestyle='--')
ax[1].fill_between(
    range(len(dataframe['fedavgm_acc'])),
    moving_average(dataframe['fedavgm_acc'] - dataframe['fedavgm_acc_std'], window),
    moving_average(dataframe['fedavgm_acc'] + dataframe['fedavgm_acc_std'], window),
    color='g', alpha=0.2
)
ax[1].plot(moving_average(dataframe['fedavgm+fedadap_acc'], window), color='b', label='FedAvgM + FedAdap', linestyle='solid')
ax[1].fill_between(
    range(len(dataframe['fedavgm+fedadap_acc'])),
    moving_average(dataframe['fedavgm+fedadap_acc'] - dataframe['fedavgm+fedadap_acc_std'], window),
    moving_average(dataframe['fedavgm+fedadap_acc'] + dataframe['fedavgm+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[1].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[1].legend(loc='lower right')
ax[1].set_title('(b)')
ax[1].set_xticks(range(0, 51, 10))
ax[1].set_xticklabels(range(0, 51, 10), rotation=45)
ax[1].set_yticks(range(20, 55, 5))
# ax[1].set_xlabel('Rounds')
ax[1].grid(True)
# ax[1].set_ylabel('Accuracy (%)')

# FedProx plot
ax[2].plot(moving_average(dataframe['fedprox_acc'], window), color='g', label='FedProx', linestyle='--')
ax[2].fill_between(
    range(len(dataframe['fedprox_acc'])),
    moving_average(dataframe['fedprox_acc'] - dataframe['fedprox_acc_std'], window),
    moving_average(dataframe['fedprox_acc'] + dataframe['fedprox_acc_std'], window),
    color='g', alpha=0.2
)
ax[2].plot(moving_average(dataframe['fedprox+fedadap_acc'], window), color='b', label='FedProx + FedAdap', linestyle='solid')
ax[2].fill_between(
    range(len(dataframe['fedprox+fedadap_acc'])),
    moving_average(dataframe['fedprox+fedadap_acc'] - dataframe['fedprox+fedadap_acc_std'], window),
    moving_average(dataframe['fedprox+fedadap_acc'] + dataframe['fedprox+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[2].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[2].legend(loc='lower right')
ax[2].set_title('(c)')
ax[2].set_xticks(range(0, 111, 20))
ax[2].set_xticklabels(range(0, 111, 20), rotation=45)
ax[2].set_yticks(range(20, 55, 5))
# ax[2].set_xlabel('Rounds')
ax[2].grid(True)
# ax[2].set_ylabel('Accuracy (%)')
# +++++++++++++++++++++++
# FedLaw plot
ax[3].plot(moving_average(dataframe['fedlaw_acc'], window), color='g', label='FedLaw', linestyle='--')
ax[3].fill_between(
    range(len(dataframe['fedlaw_acc'])),
    moving_average(dataframe['fedlaw_acc'] - dataframe['fedlaw_acc_std'], window),
    moving_average(dataframe['fedlaw_acc'] + dataframe['fedlaw_acc_std'], window),
    color='g', alpha=0.2
)
ax[3].plot(moving_average(dataframe['fedlaw+fedadap_acc'], window), color='b', label='FedLaw + FedAdap', linestyle='solid')
ax[3].fill_between(
    range(len(dataframe['fedlaw+fedadap_acc'])),
    moving_average(dataframe['fedlaw+fedadap_acc'] - dataframe['fedlaw+fedadap_acc_std'], window),
    moving_average(dataframe['fedlaw+fedadap_acc'] + dataframe['fedlaw+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[3].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[3].legend(loc='lower right')
ax[3].set_title('(d)')
ax[3].set_xticks(range(0, 51, 10))
ax[3].set_xticklabels(range(0, 51, 10), rotation=45)
ax[3].set_yticks(range(20, 55, 5))
# ax[3].set_xlabel('Rounds')
ax[3].grid(True)
# ax[3].set_ylabel('Accuracy (%)')

# FedEx plot
ax[4].plot(moving_average(dataframe['fedex_acc'], window), color='g', label='FedEx', linestyle='--')
ax[4].fill_between(
    range(len(dataframe['fedex_acc'])),
    moving_average(dataframe['fedex_acc'] - dataframe['fedex_acc_std'], window),
    moving_average(dataframe['fedex_acc'] + dataframe['fedex_acc_std'], window),
    color='g', alpha=0.2
)
ax[4].plot(moving_average(dataframe['fedavg+fedadap_acc'], window), color='b', label='FedAvg + FedAdap', linestyle='solid')
ax[4].fill_between(
    range(len(dataframe['fedavg+fedadap_acc'])),
    moving_average(dataframe['fedavg+fedadap_acc'] - dataframe['fedavg+fedadap_acc_std'], window),
    moving_average(dataframe['fedavg+fedadap_acc'] + dataframe['fedavg+fedadap_acc_std'], window),
    color='b', alpha=0.2
)
ax[4].axhline(y=target_acc, color='red', label='Target Accuracy', linestyle="dashed", linewidth=2)
ax[4].legend(loc='lower right')
ax[4].set_title('(e)')
ax[4].set_xticks(range(0, 51, 10))
ax[4].set_xticklabels(range(0, 51, 10), rotation=45)
ax[4].set_yticks(range(20, 55, 5))
# ax[4].set_xlabel('Rounds')
ax[4].grid(True)
# ax[4].set_ylabel('Accuracy (%)')

# fig.tight_layout(pad=2.0)

fig.supylabel(' Accuracy (%)')
fig.supxlabel(' Rounds')

# plt.show()
# fig.savefig(os.path.join(PLOTS_DIR,pgf_filename),dpi= 300,bbox_inches='tight')

fig.savefig(os.path.join(PLOTS_DIR,png_filename),dpi= 300,bbox_inches='tight')
fig.savefig(os.path.join(PLOTS_DIR,pdf_filename),dpi= 300,bbox_inches='tight')